# Availability Groups

Setting up availabilty groups is hard

with dbatools it is a little easier :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [30]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers
We have a connection to the containers


**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [8]:
Get-DbaDatabase -SqlInstance $SQL2 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 8d2d018abf91
Database     : AdventureWorks2017
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 8d2d018abf91
Database     : Northwind
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 8d2d018abf91
Database     : pubs
Status       : Dropped




# Containers

dbatools is not able to create availability groups in containers, so we will create an availability group below using T-SQL. If you are not using containers you can create an availability group with dbatools like this

````
$params = @{
    Primary = $SqlInstances[0]
    PrimarySqlCredential =  $SqlCredential 
    Secondary =   $SqlInstances[1]
    SecondarySqlCredential = $SqlCredential 
    Name = 'TheBeard_AG'
    Database = 'pubs','NorthWind','AdventureWorks2017'
    ClusterType = "None"
    SeedingMode = "Automatic"
    FailoverMode = "Manual"
    Confirm = $false
 }
 
# execute the command
 New-DbaAvailabilityGroup @params
 ````
 
There are many options that you can use here. Read the docs or get our book [beard.media\book]([beard.media\book)

Lets create an availability group to use with this notebook

First we shall create a master key on each instance

In [9]:
New-DbaDbMasterKey -SqlInstance $SQL1 -Credential $SqlCredential -Confirm:$false
New-DbaDbMasterKey -SqlInstance $SQL2 -Credential $SqlCredential -Confirm:$false


ComputerName        : localhost
InstanceName        : MSSQLSERVER
SqlInstance         : ce9a5ca3600e
Database            : master
CreateDate          : 09/03/2020 10:48:59
DateLastModified    : 09/03/2020 10:48:59
IsEncryptedByServer : True

ComputerName        : localhost
InstanceName        : MSSQLSERVER
SqlInstance         : 8d2d018abf91
Database            : master
CreateDate          : 09/03/2020 10:48:59
DateLastModified    : 09/03/2020 10:48:59
IsEncryptedByServer : True




# Certificate

Next we will create a certificate for the endpoints on SQL1, back it up and restore it on SQL2. You can do this for any certificate that you require by the way :-)

In [10]:
New-DbaDbCertificate -SqlInstance $sql1 -Name hadr_cert -Subject hadr_cert 

$EncryptionPassword = ConvertTo-SecureString -AsPlainText "Password4567!!" -force
Backup-DbaDbCertificate -SqlInstance $SQL1 -Suffix $null -Certificate hadr_cert -Path '/var/opt/mssql/backups/' -EncryptionPassword $EncryptionPassword 



ComputerName                 : localhost
InstanceName                 : MSSQLSERVER
SqlInstance                  : ce9a5ca3600e
Database                     : master
Name                         : hadr_cert
Subject                      : hadr_cert
StartDate                    : 09/03/2020 00:00:00
ActiveForServiceBrokerDialog : False
ExpirationDate               : 09/03/2025 00:00:00
Issuer                       : hadr_cert
LastBackupDate               : 01/01/0001 00:00:00
Owner                        : dbo
PrivateKeyEncryptionType     : MasterKey
Serial                       : 57 f7 db 5f ff 89 8d 97

Certificate  : hadr_cert
ComputerName : localhost
Database     : master
InstanceName : MSSQLSERVER
Key          : /var/opt/mssql/backups/\hadr_cert.pvk
Path         : /var/opt/mssql/backups/\hadr_cert.cer
SqlInstance  : ce9a5ca3600e
Status       : Success




Now we will use `Get-DbaFile` to ge the certificate path and then restore the certificate to SQL2
This will prompt you to ask for input. If you did not wnat this, you can use `-Force` I am also showing verbose output here. All dbatools commands can output verbose information if you use the `-Verbose` switch

In [11]:
$Certificate = (Get-DbaFile -SqlInstance $sql2 -Path '/var/opt/mssql/backups/' -FileType cer).FileName
Restore-DbaDbCertificate -SqlInstance $SQL2 -Path $Certificate -DecryptionPassword $EncryptionPassword  -Verbose


VERBOSE: [19:20:17][Restore-DbaDbCertificate] Processing /var/opt/mssql/backups/\hadr_cert.cer

Confirm
Are you sure you want to perform this action?
Performing the operation "Importing Certificate" on target "hadr_cert on localhost,15593".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


VERBOSE: [19:20:24][Restore-DbaDbCertificate] Creating Certificate: hadr_cert
VERBOSE: [19:20:24][Restore-DbaDbCertificate] Full certificate path: \var\opt\mssql\backups\\hadr_cert.cer
VERBOSE: [19:20:24][Restore-DbaDbCertificate] Private key: \var\opt\mssql\backups\\hadr_cert.pvk

ComputerName                 : localhost
InstanceName                 : MSSQLSERVER
SqlInstance                  : 8d2d018abf91
Database                     : master
Name                         : hadr_cert
Subject                      : hadr_cert
StartDate                    : 09/03/2020 00:00:00
ActiveForServiceBrokerDialog : True
ExpirationDate               : 09/03/2025 00:00:00
Issuer                       : hadr_cert
LastBackupDate               : 01/01/0001 00:00:00
Owner                        : dbo
PrivateKeyEncryptionType     : MasterKey
Serial                       : 57 f7 db 5f ff 89 8d 97




In [ ]:
# Endpoints
Now that we have the certificates on each instance, we can create the endpoints using the certificate and start them

In [12]:
New-DbaEndpoint -SqlInstance $sql1 -Name hadr_endpoint -Port 5022 -Certificate hadr_cert 
New-DbaEndpoint -SqlInstance $sql2 -Name hadr_endpoint -Port 5023 -Certificate hadr_cert

Start-DbaEndpoint -SqlInstance $SQL1 -EndPoint hadr_endpoint
Start-DbaEndpoint -SqlInstance $SQL2 -EndPoint hadr_endpoint


PowerShell credential request
Enter your credentials.


Password for user -:  




ComputerName    : localhost
InstanceName    : MSSQLSERVER
SqlInstance     : ce9a5ca3600e
ID              : 65536
Name            : hadr_endpoint
Port            : 5022
EndpointState   : Stopped
EndpointType    : DatabaseMirroring
Owner           : sqladmin
IsAdminEndpoint : False
Fqdn            : TCP://BEARDXPS:5022
IsSystemObject  : False

ComputerName    : localhost
InstanceName    : MSSQLSERVER
SqlInstance     : 8d2d018abf91
ID              : 65536
Name            : hadr_endpoint
Port            : 5023
EndpointState   : Stopped
EndpointType    : DatabaseMirroring
Owner           : sqladmin
IsAdminEndpoint : False
Fqdn            : TCP://BEARDXPS:5023
IsSystemObject  : False

ComputerName    : localhost
InstanceName    : MSSQLSERVER
SqlInstance     : ce9a5ca3600e
ID              : 65536
Name            : hadr_endpoint
Port            : 5022
EndpointState   : Started
EndpointType    : DatabaseMirroring
Owner           : sqladmin
IsAdminEndpoint : False
Fqdn            : TCP://BEARD

# Create the Availability Group

we have to use T-SQL to get the right information only because we are using containers

In [13]:

$query = @"
CREATE AVAILABILITY GROUP [TheBeard_AG]
    WITH (CLUSTER_TYPE = NONE)
    FOR REPLICA ON
        N'$($sql1.ComputerNamePhysicalNetBIOS)' WITH (
            ENDPOINT_URL = N'tcp://dbatools_SQL2019_1:5022',
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
                    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    ),
        N'$($sql2.ComputerNamePhysicalNetBIOS)' WITH ( 
		    ENDPOINT_URL = N'tcp://dbatools_SQL2019-1_1:5023', 
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
		    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    );
		
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql1 -Query $query

$query = @"
ALTER AVAILABILITY GROUP [TheBeard_AG] JOIN WITH (CLUSTER_TYPE = NONE);
		 
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql2 -Query $query


we can check that the availability groups on an instance using `Get-DbaAvailabilityGroup`

In [14]:
Get-DbaAvailabilityGroup -SqlInstance $sql1 


ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : ce9a5ca3600e
LocalReplicaRole           : Primary
AvailabilityGroup          : TheBeard_AG
PrimaryReplica             : ce9a5ca3600e
ClusterType                : None
DtcSupportEnabled          : False
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {8d2d018abf91, ce9a5ca3600e}
AvailabilityDatabases      : {}
AvailabilityGroupListeners : {}




Excellent, we have an availablity group - Lets check the replicas

In [15]:
Get-DbaAgReplica -SqlInstance $sql1


ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : ce9a5ca3600e
AvailabilityGroup          : TheBeard_AG
Name                       : 8d2d018abf91
Role                       : Secondary
ConnectionState            : Connected
RollupSynchronizationState : NotSynchronizing
AvailabilityMode           : AsynchronousCommit
BackupPriority             : 50
EndpointUrl                : tcp://dbatools_SQL2019-1_1:5023
SessionTimeout             : 10
FailoverMode               : Manual
ReadonlyRoutingList        : {}

ComputerName               : localhost
InstanceName               : MSSQLSERVER
SqlInstance                : ce9a5ca3600e
AvailabilityGroup          : TheBeard_AG
Name                       : ce9a5ca3600e
Role                       : Primary
ConnectionState            : Connected
RollupSynchronizationState : NotSynchronizing
AvailabilityMode           : AsynchronousCommit
BackupPriority             : 50
EndpointUrl          

We have two replicas - Right now they are not synchronising because there are no databases, You can use this command to see the rol up status for the replicas

Let's check the databases

In [16]:
Get-DbaAgDatabase -SqlInstance $SQL1

There are no databases - Lets add the databases

# Recovery Mode

We need ot set the recovery mode of the databases to full

In [19]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeSystem | Set-DbaDbRecoveryModel -RecoveryModel Full -Confirm:$false


ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : AdventureWorks2017
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:01
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00

ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : Northwind
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:01
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00

ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : pubs
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:02
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00




# Backup

Of course, having set the recovery model to full we need to back up the databases otherwise they are in PSUDOSIMPLE mode. You can see this. If we check the recovery model, it will show full

In [21]:
Get-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs


ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : AdventureWorks2017
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:01
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00

ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : Northwind
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:01
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00

ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : ce9a5ca3600e
Name           : pubs
Status         : Normal
IsAccessible   : True
RecoveryModel  : Full
LastFullBackup : 21/12/2019 14:42:02
LastDiffBackup : 21/12/2019 14:42:00
LastLogBackup  : 01/01/0001 00:00:00




but if we use `Test-DbaDbRecoveryModel` you can see that the *Actual* recovery model is SIMPLE

In [24]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs


ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : AdventureWorks2017
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : SIMPLE

ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : Northwind
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : SIMPLE

ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : pubs
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : SIMPLE




If we backup the databases (like we did in the BackupsandRestores notebook

In [26]:
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Full -IgnoreFileChecks
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Log -IgnoreFileChecks


SqlInstance  Database           Type TotalSize DeviceType Start                   Duration End
-----------  --------           ---- --------- ---------- -----                   -------- ---
ce9a5ca3600e AdventureWorks2017 Full 207.09 MB Disk       2020-03-09 11:07:58.000 00:00:02 2020-03…
ce9a5ca3600e Northwind          Full 6.71 MB   Disk       2020-03-09 11:08:01.000 00:00:01 2020-03…
ce9a5ca3600e pubs               Full 4.52 MB   Disk       2020-03-09 11:08:02.000 00:00:00 2020-03…
ce9a5ca3600e AdventureWorks2017 Log  84.00 KB  Disk       2020-03-09 11:08:03.000 00:00:00 2020-03…
ce9a5ca3600e Northwind          Log  80.00 KB  Disk       2020-03-09 11:08:03.000 00:00:00 2020-03…
ce9a5ca3600e pubs               Log  80.00 KB  Disk       2020-03-09 11:08:03.000 00:00:00 2020-03…



and test our recovery model again

In [27]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs


ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : AdventureWorks2017
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : FULL

ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : Northwind
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : FULL

ComputerName            : ce9a5ca3600e
InstanceName            : MSSQLSERVER
SqlInstance             : ce9a5ca3600e
Database                : pubs
ConfiguredRecoveryModel : FULL
ActualRecoveryModel     : FULL




All our databases are now in FULL recovery :-)

Let's add them to the Availability Group

In [31]:
Add-DbaAgDatabase -SqlInstance $sql1 -AvailabilityGroup TheBeard_AG -Database AdventureWorks2017, Northwind, pubs -Secondary $sql2 -SeedingMode Automatic


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False



ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : 8d2d018abf91
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : Synchronizing
IsFailoverReady      : False
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : pubs
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False




Let's have a look at them


In [32]:
Get-DbaAgDatabase -SqlInstance $SQL1 


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : AdventureWorks2017
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : Northwind
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False

ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : ce9a5ca3600e
AvailabilityGroup    : TheBeard_AG
Replica              : localhost
Name                 : pubs
SynchronizationState : Synchronized
IsFailoverReady      : True
IsJoined             : True
IsSuspended          : False




You can look in SSMS if you like :-)

![AG](.\images\ag.png )